In [1]:
#Imports + Load Data + Load SVD Model
import pandas as pd
import pickle
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/Electronics.csv.gz"
MODEL_PATH = "/content/drive/MyDrive/final_svd_model.pkl"

# Load data
df = pd.read_csv(DATA_PATH, compression="gzip")
df = df.rename(columns={
    'user_id': 'user',
    'parent_asin': 'item',
    'rating': 'rating'
})

# Load trained SVD artifacts
with open(MODEL_PATH, "rb") as f:
    model_data = pickle.load(f)

user_factors = model_data["user_factors"]
item_factors = model_data["item_factors"]
users = model_data["users"]
items = model_data["items"]

print("Data & SVD model loaded successfully")


Mounted at /content/drive
Data & SVD model loaded successfully


In [2]:
#Popularity Model (Cold‑Start Component)
popularity = (
    df.groupby('item')['rating']
    .mean()
    .sort_values(ascending=False)
)

print("Popularity model ready")


Popularity model ready


In [3]:
#Hybrid Recommendation Function
def hybrid_recommend(user_id, top_n=10, alpha=0.7):
    """
    alpha → weight for CF
    (1 - alpha) → weight for popularity
    """
    if user_id not in users:
        # Cold start user → popularity only
        return popularity.head(top_n).index.tolist()

    user_idx = users.get_loc(user_id)

    # CF scores using SVD
    cf_scores = np.dot(user_factors[user_idx], item_factors)

    cf_series = pd.Series(cf_scores, index=items)

    # Combine CF + Popularity
    hybrid_scores = (
        alpha * cf_series +
        (1 - alpha) * popularity.reindex(items).fillna(popularity.mean())
    )

    return hybrid_scores.sort_values(ascending=False).head(top_n).index.tolist()


In [4]:
#Test Hybrid Recommender
sample_user = users[0]
hybrid_recommend(sample_user, top_n=10)



['B09XKM8TGQ',
 'B0BZTF3GPM',
 'B087WMG58R',
 'B0C6B4GFDK',
 'B08QF4L3X4',
 'B0BWNF2L2S',
 'B08MC7M6CR',
 'B07Y9MNQJB',
 'B096Y7FVBX',
 'B083GXNGTG']

In [5]:
#Make sure your trained SVD model file exists for deployment.
import os

MODEL_PATH = "/content/drive/MyDrive/final_svd_model.pkl"
os.path.exists(MODEL_PATH)


True